In [1]:
pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 25.3 MB/s 


In [2]:
from Crypto.Cipher import AES
from binascii import unhexlify, hexlify, b2a_base64, a2b_base64
from secrets import token_bytes
from random import choice, randint
from urllib.parse import quote_from_bytes

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("drive/My Drive/Courses/Industrial Internet of Things/Final assignment/algorithms/")

Mounted at /content/drive


In [4]:
def hex_to_base64(hex_str):

  """
  Converts hexadecimal strings to base64 strings
  Input: 
    hex_str: hexadecimal string
  Output: 
    base64_str: base64 string

  """
  # Converts the hexadecimal string into binary data
  bin_str = unhexlify(hex_str)

  # Converts the binary data into base64 string
  base64_str = b2a_base64(bin_str)
  return base64_str


def fixed_xor(hex_str1, hex_str2):

  """
  Performs fixed XOR operations in two hexadecimal strings of same size
  Inputs:
    hex_str1: first hexadecimal string
    hex_str2: second hexadecimal string
  Output:

  """
  # Converts hex_str1 and hex_str2 to binary data
  bin_str1 = unhexlify(hex_str1)
  bin_str2 = unhexlify(hex_str2)

  # Iterates and performs XOR in each byte of both strings
  bin_xor = [bin_str1[i]^bin_str2[i] for i in range(len(bin_str2))]

  # Converts the result converted into hexadecimal string
  hex_xor = hexlify(bytes(bin_xor))
  return hex_xor


def plaintext_score(plaintext):

  """
  Returns a score of an english plaintext
  Input:
    plaintext: string to be analysed
  Output:
    mse: minimum square error 

  """

  # Frequencies of each character in english texts
  char_freq = {
      'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
      'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
      'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
      'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
      'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
      'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
      'y': .01974, 'z': .00074, ' ': .13000,
  }

  # Makes sure the plaintext has only lower case letters and gets its length
  text = plaintext.lower()
  n = len(text)

  # Determines the minimum square error between the expected frequencies of 
  # each character (keys) in char_freq and its current occurency. The mean 
  # square error is penalized if the plaintext contains unexpected characters
  mse = sum([(text.count(ord(char))/n - char_freq[char])**2 for char in char_freq.keys()])/n
  mse += sum([0.5*(text.count(char))/n for char in set(text) if char_freq.get(chr(char))==None])

  return mse


def single_xor_decryption(cipher):

  """
  Decrypts a XOR ciphered string, encrypted with a one character key
  Input:
    cipher: XOR ciphered string (can be both hexadecimal string or binary)
  Output:
    result: A dictionary containing the plaintext, the key and the text score

  """

  # Makes sure the cipher text is converted into binary data
  if isinstance(cipher, str):
    cipher = unhexlify(cipher)

  # Initializes the variables
  decryption_score = 1
  decryption_key = ''
  decryption_text = ''
  characters = range(1,256)

  # Performs brute force XOR decryption for 255 ASCII characters and stores
  # the best result
  for char in characters:

    # XOR decryption
    plaintext = bytes([byte^char for byte in cipher])

    # Gets the plaintext score
    score = plaintext_score(plaintext)

    # Updates the stored values if a better score is obtained
    if decryption_score > score:
      decryption_score = score
      decryption_key = chr(char)
      decryption_text = plaintext

  # Returns the result
  result = {'score': decryption_score, 'key': decryption_key, 'text': decryption_text}
  return result


def repeating_xor_encryption(plaintext, key):

  """
  XOR encrypts a plaintext
  Inputs:
    plaintext: string to be encrypted
    key: key for XOR encryption
  Outuput
    hexadecimal cipher string

  """

  # Converts the plaintext and the key to bytes data
  plaintext_bytes = plaintext.encode()
  key_bytes = key.encode()

  # Initializes the ciphertext and the key
  ciphertext = []
  key_position = 0

  # Iterates over each byte within the plaintext
  for byte in plaintext_bytes:

    # XOR encrypts the plaintext byte and the corresponding key byte
    ciphertext.append((byte ^ key_bytes[key_position]) & 255)

    # Updates the key to be used on next encryption
    if key_position + 1 == len(key):
      key_position = 0
    else:
      key_position += 1

  # Returns the XOR encrypted ciphertext and a hexadecimal string
  return hexlify(bytes(ciphertext))


def hamming_distance(string1_bytes, string2_bytes):
  
  """
  Calculates the hamming distance between two byte strings
  Inputs:
    string1_bytes = first string converted into bytes
    string2_bytes = second string converted into bytes
  Output:
    distance: hamming distance

  """
  
  # Initializes the variable that will store the hamming distance
  distance = 0

  # Iterates over each byte
  for byte in range(len(string1_bytes)):
      
    # Iterates over each bit within each byte
    for bit in range(8):

      # Gets the bit of each string
      string1_bit = (string1_bytes[byte] >> bit) & 1
      string2_bit = (string2_bytes[byte] >> bit) & 1

      # Compares both strings and updates the hamming distance
      if string1_bit != string2_bit:
        distance += 1

  # Returns the hamming distance
  return distance


def repeating_xor_decryption(ciphertext, num_strings):

  """
  Decrypts a XOR ciphered string, encrypted with one or more character keys
  Input:
    ciphertext: XOR ciphered string
    num_strings = number of string blocks used for average hamming calculation
  Output:
    result: A dictionary containing the plaintext, the key and the text score
    
  """

  # Initializes the variable that will store the best key size
  best_keysize = {'keysize': 0, 'distance':256, 'strings': []}

  # Tries keys of different sizes
  for keysize in range(2, 41):

    # Breks the ciphertext into num_strings blocks of keysize bytes
    strings = []
    for string in range(num_strings):
      strings.append(bytes(ciphertext[string*keysize:(string+1)*keysize]))
    
    # Computes the normalized hamming distance
    normalized_dist = average_hamming_distances(strings)/keysize

    # Updates best result if a smaller hamming distance is obtained
    if normalized_dist < best_keysize['distance']:
      best_keysize['distance'] = normalized_dist
      best_keysize['keysize'] = keysize
      best_keysize['strings'] = strings

  # Determines the encryption key
  key = []
  for byte in range(best_keysize['keysize']):
    
    # Gets the plaintext bytes encrypted with the same key byte
    same_key_bytes = []
    ith_byte = byte
    while ith_byte < len(ciphertext):
        same_key_bytes.append(ciphertext[ith_byte])
        ith_byte += best_keysize["keysize"]

    # Gets the key byte and the corresponding decrypted bytes
    key.append(single_xor_decryption(bytes(same_key_bytes)))

  # Gets the final plaintext 
  plaintext = []
  for char in range(len(ciphertext)):
    plaintext.append(chr(key[char%best_keysize['keysize']]["text"][char//best_keysize['keysize']]))
  
  return ''.join(plaintext)


def average_hamming_distances(strings):

  """
  Determines the average hamming distance of two or more byte strings of
  same size
  Input:
    strings: list containing the byte strings
  Output:
    average hamming distance

  """

  # Gets the number of strings and initializes the variables for computing 
  # the average hamming distance
  num_strings = len(strings)
  sum_distance = 0
  num_distance = 0

  # Computes the hamming distance for each combination of two strings passed
  # the input argurment  
  for string1 in range(num_strings):
      for string2 in range(string1+1, num_strings):
          sum_distance += hamming_distance(strings[string1], strings[string2])
          num_distance += 1

  # Returns the average hamming distance
  return sum_distance/(num_distance)

In [25]:
def pkcs7_padding(plaintext, num_bytes):

  """
  Pads a block of plaintext according to PKCS#7 padding scheme
  Inputs:
    plaintext: hexadecimal string or plaitext already converted to bytes
    num_byes: number of bytes used for padding
  Output:
    padded plaintext  
  """

  # If the plaintext is a hexadecimal string, converts it into bytes data
  if isinstance(plaintext, str):
    plaintext = unhexlify(plaintext)
  # Checks the number of bytes required for padding
  num_bytes_padding = num_bytes - len(plaintext)
  # Creates the padding
  padding = bytes(chr(num_bytes_padding),'ascii')*num_bytes_padding
  # Adds the padding to the plaintext and returns it
  return plaintext + padding


def xor_bytes(bytes1, bytes2):

  """
  Performs XOR operation of two blocks of bytes of same size
  Inputs:
    bytes1 = first block of bytes
    bytes2 = second block of bytes
  Output:
    XOR combined bytes

  """
  
  # Performs XOR operation with both bytes and return the result
  xor = [bytes1[i]^bytes2[i] for i in range(len(bytes1))]
  return xor


def ECB_encryption(plaintext, key):

  """
  ECB encrypts a plaintext against a provided key
  Inputs:
    plaintext: plaintext (bytes) that is going to be encrypted
    key: key (bytes) to be used for encryption
  Output:
    ciphertext: ciphertext (bytes) with the encrypted data
  """

  # Checks the block size based on the length of the key and initializes the
  # ciphertext as an empty bytes data
  block_size = len(key)
  ciphertext = b""

  # Iterates over each block within the plaintext
  total_bytes = len(plaintext)
  for byte in range(0, len(plaintext), block_size):

    # Extracts the plaintext block, padding it if necessary
    if byte + block_size <= total_bytes:
      plaintext_block = plaintext[byte:byte+block_size]
    else:
      plaintext_block = pkcs7_padding(plaintext[byte:total_bytes], block_size)

    # Encrypts the XORed plaintext using the given key and updtates the IV
    obj = AES.new(key, AES.MODE_ECB)
    ciphertext += obj.encrypt(bytes(plaintext_block))

  # Returns the ciphertext
  return ciphertext


def ECB_decryption(ciphertext, key):

  """
  ECB decrypts the plaintext
  Inputs:
    ciphertext: the ciphertext (bytes) to be decrypted
    key: the key (bytes) used for decryption
  Output:
    plaintext: the decrypted (bytes) plaintext
  """

  # Checks the block size for decryption and initilializes the plaintext
  block_size = len(key)
  plaintext = b""

  # Iterates over each block within the ciphertext
  for byte in range(0, len(ciphertext), block_size):

    # Extracts the ciphertext block and decrypts it
    ciphertext_block = ciphertext[byte:byte+block_size]
    obj = AES.new(key, AES.MODE_ECB)

    # Appends the plaintext block to the plaintext variable
    plaintext += obj.decrypt(bytes(ciphertext_block))

  return plaintext


def CBC_encryption(plaintext, key, iv):

  """
  CBC encrypts a plaintext
  Inputs:
    plaintext: the plaintext to be encrypted, converted to bytes
    key: the key used for CBC encryption, converted to bytes
    iv: initialization vector, converted to bytes
  Output:
    ciphertext: CBC encrypted plaintext
  """

  # Checks the block size based on the length of the IV and initializes the
  # ciphertext as an empty bytes data
  block_size = len(iv)
  ciphertext = b""

  # Iterates over each block within the plaintext
  total_bytes = len(plaintext)
  for byte in range(0, len(plaintext), block_size):

    # Extracts the plaintext block and XORd's it with the IV
    if byte + block_size <= total_bytes:
      plaintext_block = plaintext[byte:byte+block_size]
    else:
      plaintext_block = pkcs7_padding(plaintext[byte:total_bytes], block_size)
    xored_plaintext = [plaintext_block[j]^iv[j] for j in range(block_size)]

    # Encrypts the XORed plaintext using the given key and updtates the IV
    obj = AES.new(key, AES.MODE_ECB)
    iv = obj.encrypt(bytes(xored_plaintext))
    ciphertext += iv

  # Returns the ciphertext
  return ciphertext


def CBC_decryption(ciphertext, key, iv):

  """
  Decrypts a CBC encrypted plaintext
  Inputs:
    plaintext: the plaintext to be decrypted, converted to bytes
    key: the key used for CBC decryption, converted to bytes
    iv: initialization vector, converted to bytes
  Output:
    Plaintext converted to bytes
  """

  # Checks the block size based on the length of the IV and initializes the
  # plaintext as an empty bytes data
  block_size = len(iv)
  plaintext = b""

  # Iterates over each block within the ciphertext
  for byte in range(0, len(ciphertext), block_size):

    # Extracts the ciphertext block
    ciphertext_block = ciphertext[byte:byte+block_size]

    # Decrypts the ciphertext block and adds it to the plaintext
    obj = AES.new(key, AES.MODE_ECB)
    decrypted_ciphertext = obj.decrypt(bytes(ciphertext_block))
    list = [decrypted_ciphertext[j]^iv[j] for j in range(block_size)]
    plaintext += bytes(list)

    # Updates the IV for next decryption
    iv = ciphertext_block

  # Returns the plaintext as bytes
  return plaintext


def random_ECB_CBC_encryption(plaintext, block_size):

  """
  Randomly ECB or CBC encrypts a plaintext
  Inputs:
    plaintext: the plaintext (bytes) to be encrypted
    block_size: the size of the block used for encryption
  Output:
    ciphertext: the plaintext encrypted
    cipher_mode: the randomly chosen mode (used for oracle validation)
  """

  # Generates 5 to 10 random bytes to append at the beggining and ending of
  # the plaintext
  header = token_bytes(randint(5,10))
  footer = token_bytes(randint(5,10))
  plaintext = header + plaintext + footer

  # Randomly chooses the encryption mode
  encryption_mode = choice(["ECB","CBC"])

  # Generates a block_size bytes random key
  key = token_bytes(block_size)

  # Encrypts the ciphertext according to the randomly chosen mode
  if encryption_mode == "ECB":

    # Performs ECB encryption
    ciphertext = ECB_encryption(plaintext, key)

  else: 

    # Generates a random block_size bytes IV
    iv = token_bytes(block_size)

    # Performs CBC encryption
    ciphertext = CBC_encryption(plaintext, key, iv)

  return ciphertext, encryption_mode


def ECB_CBC_oracle(ciphertext, num_bytes):

  """
  Determines the encryption mode (ECB or CBC) used in a ciphertext
  Inputs:
    ciphertext: the ciphertext (into bytes) to be analysed
    num_bytes: the size of the block used for encryption
  Output:
    mode: ECB or CBC
  """

  # Divides the ciphertext into blocks of num_bytes bytes
  grouped_bytes = [ciphertext[n:n+num_bytes] for n in range(0, len(ciphertext), num_bytes)]

  # Checks how many blocks of bytes are repeated
  num_repetitions = len(grouped_bytes) - len(set(grouped_bytes))

  # Determines the encryption mode if there are repeated blocks or not
  if num_repetitions == 0:
    mode = "CBC"
  else:
    mode = "ECB"

  return mode


def byte_at_a_time_ECB_decryption(plaintext, key, prefix=b""):

  """
  Appends an unknown string to a plaintext and encrypts it under ECB
  Inputs:
    plaintext: the plaintext converted to bytes
    key: the key (in bytes) used for ECB encryption
    prefix: Optional argument for appending a string before the plaintext
  Output:
    ciphertext: the ECB encrypted ciphertext
  """

  # Converts the unkown string to binary data
  unknown_str = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg" \
                "aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq" \
                "dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg" \
                "YnkK"
  unknown_str = a2b_base64(unknown_str)

  # Appends the string to the plaintext and encrypts it
  plaintext = prefix + plaintext + unknown_str
  ciphertext = ECB_encryption(plaintext, key)

  return ciphertext


def profile_parsing(profile):

  """
  Parses a user profile, creating key:value pairs with user data
  Inputs:
    profile: string with user data, separated with characters '&' and '='.
      Example: "info1=data1&info2=data2&info3=data3"
  Output:
    dictinary: dictionary with the user data
  """

  # Splits the string with character "&"
  profile = profile.split("&")

  # Splits the strings and creates a dictionary with the key-value pairs
  dictionary = dict()
  for line in profile:
    key, value = line.split("=")
    dictionary[key] = value
  
  return dictionary


def user_profile(email):

  """
  Encodes a user profile
  Inputs:
    email: user e-mail
  Output:
    encoded_profile: encoded user data, converted to bytes
  """

  # Makes sure the email does not contain characters '&' and '='
  email = email.replace("&", "")
  email = email.replace("=", "")

  # Encodes the user email with other data and converts it to bytes
  encoded_profile = "email=" + email + "&uid=10" + "&role=user"  
  return encoded_profile.encode()


def profile_encoding(profile, key):

  """
  # Encrypts a user profile with the ECB mode
  Input:
    profile: string with the user data
    key: key to be used for encryption
  Output:
    encoded_user_profile: ciphertext resulting from the encryption
  """

  # Encrypts the user profile under ECB mode
  encoded_user_profile = ECB_encryption(profile, key)
  return encoded_user_profile


def profile_decoding(encoded_profile, key):

  """
  # Decrypts a user profile encrypted with the ECB mode
  Input:
    encoded_profile: ECB encrypted user data
    key: key to be used for decryption
  Output:
    encoded_user_profile: plaintext resulting from the decryption
  """

  # Decrypts the user profile data encrypted under ECB mode
  decoded_user_profile = ECB_decryption(encoded_profile, key)
  return decoded_user_profile


def is_pkcs7_valid(plaintext, block_size):

  """
  Checks if a PKCS7 padding is valid
  Inputs:
    plaintext: padded plaintext (bytes)
    block_size: the size used for padding
  """

  # If the plaintext size is not multiple of the block size, returns False
  if len(plaintext) % block_size != 0:
    return False
  else:
    last_char = plaintext[block_size-1]
    # If the last_char is greater than block size, assumes the padding was
    # not necessary
    if last_char < block_size:
      for char in plaintext[block_size-last_char:block_size]:
        # If the padding is performed with different characters, returns False
        if last_char != char:
          return False
  # Returns true otherwise
  return True


def encryption(plaintext, key, iv):

  """
  CBC encrypts a user profile, adding additional data before and after it.
  Inputs:
    plaintext: the plaintext (bytes) to be encrypted
    key: the key to be used for encryption
    iv: the initialization vector to be used for encryption
  Output:
    ciphertext: the ciphertext (bytes) after CBC encryption
  """

  # Adds the strings before and after the plaintext
  pre = b"comment1=cooking%20MCs;userdata="
  pos = b";comment2=%20like%20a%20pound%20of%20bacon"

  # Quotes out the special characters and encrypts it
  plaintext = pre + plaintext + pos
  ciphertext = CBC_encryption(quote_from_bytes(plaintext).encode(), key, iv)
  # plaintext = pre + quote_from_bytes(plaintext).encode() + pos
  # ciphertext = CBC_encryption(plaintext, key, iv)

  return ciphertext


def check_admin(ciphertext, key, iv):

  """
  Decrypts a CBC encrypted user profile and checks if the user is admin
  Inputs:
    ciphertet: the ciphertext (bytes) with user data
    key: the key to be used for decryption
    iv: the initialization vector to be used for decryption
  Output:
    The plaintext and a boolean value informing if user is admin or not
  """

  # Decrypts the plaintext
  plaintext = CBC_decryption(ciphertext, key, iv)

  # Checks if user is admin
  if b";admin=true;" in plaintext:
    return plaintext, True
  else:
    return plaintext, False


In [6]:
# Exercise 1

hex_string = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706"\
             "f69736f6e6f7573206d757368726f6f6d"
print(hex_to_base64(hex_string))

# Exercise 2

hex_string1 = "1c0111001f010100061a024b53535009181c"
hex_string2 = "686974207468652062756c6c277320657965"
print(fixed_xor(hex_string1, hex_string2))

# Exercise 3

hex_string = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393"\
             "b3736"            
print(single_xor_decryption(hex_string))

# Exercise 4

with open('04.txt', 'r') as file:
  ciphertext = file.readlines()
  best_plaintext = {'score': 1}
  best_ciphertext = []
  for ciphertext_row in ciphertext:
    plaintext = single_xor_decryption(ciphertext_row.replace('\n',''))
    if plaintext['score'] < best_plaintext['score']:
      best_plaintext = plaintext
      best_ciphertext = ciphertext_row
print(best_plaintext)
print(best_ciphertext)

# Exercise 5

string = "Burning 'em, if you ain't quick and nimble\n" \
          "I go crazy when I hear a cymbal"
key = "ICE"
print(repeating_xor_encryption(string, key))

# Exercise 6

string1 = b"this is a test"
string2 = b"wokka wokka!!!"
print(hamming_distance(string1, string2))
with open('06.txt', 'r') as file:
    b64_ciphertext = file.read().replace('\n','')
    bin_ciphertext = a2b_base64(b64_ciphertext)
    plaintext = repeating_xor_decryption(bin_ciphertext, 4)
    print(plaintext)

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t\n'
b'746865206b696420646f6e277420706c6179'
{'score': 0.01595741845181152, 'key': 'X', 'text': b"Cooking MC's like a pound of bacon"}
{'score': 0.017483136324074073, 'key': '5', 'text': b'Now that the party is jumping\n'}
7b5a4215415d544115415d5015455447414c155c46155f4058455c5b523f

b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'
37
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me

In [7]:
# Exercise 7

with open('07.txt', 'r') as file:
    b64_ciphertext = file.read().replace('\n','')
    bin_ciphertext = a2b_base64(b64_ciphertext)
    key = b"YELLOW SUBMARINE"
    obj = AES.new(key, AES.MODE_ECB)
    plaintext = obj.decrypt(bin_ciphertext)
    print(plaintext.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


In [8]:
# Exercise 8

num_bytes = 16
with open('08.txt', 'r') as file:
    hex_ciphertext = file.readlines()
    ECB_ciphertext = {"ciphertext": "", "line": 0, "repeated_blocks": 0}
    line = 0
    for hex_ciphertext_line in hex_ciphertext:
      # Splits the ciphertexts into blocks of num_bytes bytes
      ciphertext = unhexlify(hex_ciphertext_line.replace('\n',''))
      grouped_bytes = [ciphertext[n:n+num_bytes] for n in range(0, len(ciphertext), num_bytes)]
      # Checks how many blocks of bytes are repeated
      num_repetitions = len(grouped_bytes) - len(set(grouped_bytes))
      # Updates which ciphertext was ECB encrypted if a ciphertext with more 
      # repetitions of blocks of bytes is obtained
      if num_repetitions > ECB_ciphertext["repeated_blocks"]:
          ECB_ciphertext["ciphertext"] = ciphertext
          ECB_ciphertext["line"] = line
          ECB_ciphertext["repeated_blocks"] = num_repetitions
      line += 1
    print(ECB_ciphertext["ciphertext"])
    

b'\xd8\x80a\x97@\xa8\xa1\x9bx@\xa8\xa3\x1c\x81\n=\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xe2\xdd\x05/kd\x1d\xbf\x9d\x11\xb04\x85B\xbbW\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x94u\xc9\xdf\xdb\xc1\xd4e\x97\x94\x9d\x9c~\x82\xbfZ\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\x97\xa9>\xab\x8dj\xec\xd5fH\x91Tx\x9ak\x03\x08d\x9a\xf7\r\xc0oO\xd5\xd2\xd6\x9ctL\xd2\x83\xd4\x03\x18\x0c\x98\xc8\xf6\xdb\x1f*?\x9c@@\xde\xb0\xabQ\xb2\x993\xf2\xc1#\xc5\x83\x86\xb0o\xba\x18j'


In [9]:
# Exercise 9

print(pkcs7_padding(b"YELLOW SUBMARINE", 20))
print(len(pkcs7_padding(b"YELLOW SUBMARINE", 20)))

b'YELLOW SUBMARINE\x04\x04\x04\x04'
20


In [10]:
# Exercise 10

# Initializes the key and the IV
key = b"YELLOW SUBMARINE"
iv = b"\x00" * 16

with open('10.txt', 'r') as file:

  # Reads the ciphertext and converts it to bytes
  b64_ciphertext = file.read().replace('\n','')
  ciphertext = a2b_base64(b64_ciphertext)

  # Decrypts the ciphertext
  plaintext = CBC_decryption(ciphertext, key, iv)

  # Encrypts the ciphertext again
  cipher = CBC_encryption(plaintext, key, iv)

print(plaintext)
print(b2a_base64(cipher))

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

In [11]:
# Exercise 11

plaintext = b"AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"

for i in range(1024):
  ciphertext, real_mode = random_ECB_CBC_encryption(plaintext, 16)
  guessed_mode = ECB_CBC_oracle(ciphertext, 16)
  if real_mode != guessed_mode:
    print("Error")

In [12]:
# Exercise 12

# Generates a random 16 bytes key
key = token_bytes(16)

# Initializes the variables required for identification of cipher block length
num_bytes = 1
previous_first_bytes = b""
block_size = 0

# Repeats until the cipher block length is obtained
while True:

  # Encrypts a plaintext of num_bytes A's
  plaintext = b"A"*num_bytes
  ciphertext = byte_at_a_time_ECB_decryption(plaintext, key)

  # If the first bytes within the ciphertext do not change from previous 
  # iteration, the block size is then known and it leaves the while loop. 
  # Otherwise, it tries one more byte in next iteration
  if previous_first_bytes == ciphertext[0:4]:
    block_size = num_bytes - 1
    break
  else:
    previous_first_bytes = ciphertext[0:4]
    num_bytes += 1

# Checks the encryption mode
plaintext = b"A"*block_size*3
ciphertext = byte_at_a_time_ECB_decryption(plaintext, key)
encryption_mode = ECB_CBC_oracle(ciphertext, block_size)

# Initializes the variables used to determine the unknown string length
num_bytes = 1
prev_ciphertext_length = 0
unknown_string_length = 0

while True:
  # Encrypts a plaintext of num_bytes A's
  plaintext = b"A"*num_bytes
  ciphertext = byte_at_a_time_ECB_decryption(plaintext, key)

  # If the ciphertext increases, determines the unknown string length. Increases
  # the number of A's used in the plaintext otherwise
  if prev_ciphertext_length != 0 and len(ciphertext) != prev_ciphertext_length:
    unknown_string_length = len(ciphertext) - block_size - num_bytes + 1
    break
  else:
    prev_ciphertext_length = len(ciphertext)
    num_bytes += 1

# Initializes the variables used to determine the unknown plaintext
target = bytes([])
target_block = 0
target_cipher = b""

# Iterates over each targe byte within the plaintext
for target_byte in range(unknown_string_length):
 
  # Updates which plaintext block is the target
  if target_byte % block_size == 0:
    target_block = target_byte // block_size
  
  # Determines the reference ciphertext block
  # (block_size - 1) known bytes + 1 target string byte 
  known_blocks = target_block*block_size
  plaintext = b"A"*(block_size-(target_byte%block_size)-1)
  ciphertext = byte_at_a_time_ECB_decryption(plaintext, key)
  target_cipher = ciphertext[known_blocks:known_blocks+block_size]

  # Iterates over the ASCII characters in the last plaintext byte to verify 
  # which one leads to to the same target ciphertext block
  for byte in range(256):
    dict_plaintext = plaintext + target + chr(byte).encode()
    dict_ciphertext = byte_at_a_time_ECB_decryption(dict_plaintext, key)
    if target_cipher == dict_ciphertext[known_blocks:known_blocks+block_size]:
      target += chr(byte).encode()
      break

# Prints the obtained plaintext
print(target)
    

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n"


In [13]:
# Exercise 13

# Gerentes a 16 bytes random key
key = token_bytes(16)
num_bytes = len(key)
print(key)

# Encodes the user profile so that the last block is a 'user' permission we
# want to change
profile = user_profile("ggggggggggggggustavo@mail.com")
profile_bytes = [profile[n:n+num_bytes] for n in range(0, len(profile), num_bytes)]
print(profile_bytes)

# Creates an attack email with the last block corresponding the the 'admin'
# string we want to insert in the original user profile
substitution_string = pkcs7_padding(b"admin", num_bytes)
atk_email = "AAAAAAAAAA" + substitution_string.decode()
atk_profile = user_profile(atk_email)
atk_bytes = [atk_profile[n:n+num_bytes] for n in range(0, len(atk_profile), num_bytes)]
print(atk_bytes)

# Places the last block from the atk_profile into the original user_profile
encoded_profile = profile_encoding(profile, key)
encoded_atk_profile = profile_encoding(atk_profile, key)
new_encoded_profile = encoded_profile[0:48] +  encoded_atk_profile[16:32]
decoded_profile = profile_decoding(new_encoded_profile, key)
print(decoded_profile)

b'\xba.\xc5\xcff\xec\xd9\xd9`h\xaeA\xa2\xf8e\xae'
[b'email=gggggggggg', b'ggggustavo@mail.', b'com&uid=10&role=', b'user']
[b'email=AAAAAAAAAA', b'admin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b', b'&uid=10&role=use', b'r']
b'email=ggggggggggggggustavo@mail.com&uid=10&role=admin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'


In [14]:
# Exercise 14

# Generates a random size prefix and a 16 bytes random key
prefix = token_bytes(randint(0, 20))
key = token_bytes(16)

# Initializes the variables required for identification of cipher block length
previous_first_bytes = b""
previous_ciphertext = byte_at_a_time_ECB_decryption(b"", key, prefix)
block_size = 0
required_padding = 0

# Repeats until the number of bytes of the key is obtained
num_bytes = 1
while True:

  print(num_bytes)

  # Encrypts a plaintext of num_bytes A's
  plaintext = b"A"*num_bytes
  ciphertext = byte_at_a_time_ECB_decryption(plaintext, key, prefix)
  # If the ciphertext size changes, a plaintext of size equal to the block 
  # size plus one has been provided
  if len(ciphertext) != len(previous_ciphertext):
    block_size = len(ciphertext) - len(previous_ciphertext)
    padding = num_bytes - 1
    break
  else:
    num_bytes += 1

# Initializes the variables required for determining the prefix and the
# target string lengths
two_consecutive_blocks = False
num_bytes = 0

while not two_consecutive_blocks:

  # Creates a plaintex filled with A's
  plaintext = b"A"*((2*block_size)+num_bytes)
  ciphertext = byte_at_a_time_ECB_decryption(plaintext, key, prefix)
  grouped_bytes = [ciphertext[n:n+block_size] for n in range(0, len(ciphertext), block_size)]

  # When two equal blocks are obtained, the prefix was padded with A's and 
  # the block sizes of A's were also provided. Calculates the prefix length 
  # with this information. 
  for block in range(1, len(grouped_bytes)):
    if grouped_bytes[block] == grouped_bytes[block-1]:
      target_string_length = len(ciphertext) - (block+1)*block_size
      prefix_length = len(ciphertext) - target_string_length - len(plaintext)
      two_consecutive_blocks = True
  num_bytes += 1

# Determines the target string length
prefix_padding = padding - (prefix_length%block_size)
target_string_length = target_string_length - (padding - prefix_padding)

for target_byte in range(target_string_length):
 
 # Updates which plaintext block is the target
  if target_byte % block_size == 0:
    target_block = target_byte // block_size
  
  # Determines the reference ciphertext block
  # (block_size - 1) known bytes + 1 target string byte 
  known_blocks = (prefix_length+prefix_padding)%block_size + target_block*block_size
  plaintext = b"A"*(prefix_padding+block_size-(target_byte%block_size)-1)
  ciphertext = byte_at_a_time_ECB_decryption(plaintext, key, prefix)
  target_cipher = ciphertext[known_blocks:known_blocks+block_size]

  # Iterates over the ASCII characters in the last plaintext byte to verify 
  # which one leads to to the same target ciphertext block
  for byte in range(256):
    dict_plaintext = plaintext + target + chr(byte).encode()
    dict_ciphertext = byte_at_a_time_ECB_decryption(dict_plaintext, key)
    if target_cipher == dict_ciphertext[known_blocks:known_blocks+block_size]:
      target += chr(byte).encode()
      break

print(target)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n"


In [15]:
# Exercise 15

plaintext = b"ICE ICE BABY\x04\x04\x04\x04"
print(is_pkcs7_valid(plaintext, 2))

True


In [28]:
# Exercise 16

# Generates 16 bytes random key and IV
key = token_bytes(16)
iv = token_bytes(16)

# Defines the target plaintext we want to insert and a known plaintext of
# same size
target_plaintext = b";admin=true;"
known_plaintext = b"A"*(len(target_plaintext) + 8)

# Encrypts the known plaintext and confirms the user is not admin
ciphertext = encryption(known_plaintext, key, iv)
plaintext, admin = check_admin(ciphertext, key, iv)
print(admin)

# Divides the plaintext and the ciphertext into blocks of 16 bytes
grouped_bytes = [plaintext[n:n+16] for n in range(0, len(plaintext), 16)]
print(grouped_bytes)
grouped_bytes = [ciphertext[n:n+16] for n in range(0, len(ciphertext), 16)]
print(grouped_bytes)

# Modifies the ciphertext to get the admin permission
a = bytes(xor_bytes(target_plaintext, b"A"*len(target_plaintext)))
padding = 16-len(a)
b = bytes(xor_bytes(grouped_bytes[2], a + b"\x00"*padding))
grouped_bytes[2] = b
ciphertext = b''.join(grouped_bytes)

# Checks if user is admin
plaintext, admin = check_admin(ciphertext, key, iv)
grouped_bytes = [plaintext[n:n+16] for n in range(0, len(plaintext), 16)]
print(grouped_bytes)
print(admin)

False
[b'comment1%3Dcooki', b'ng%2520MCs%3Buse', b'rdata%3DAAAAAAAA', b'AAAAAAAAAAAA%3Bc', b'omment2%3D%2520l', b'ike%2520a%2520po', b'und%2520of%2520b', b'acon\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c']
[b'\x1f\xdc\xd2\x89\xee\xbf;>`Mb\\\xf9\xd82F', b'\x9c\xb6\x9a\xc7\x11\x03g\x9cw\x0c\x9d\xe4\xcf\x87sz', b'c\xc6\xac\xb0qj\xe5\xb4_\x07\xad(Ve)\xd5', b'\x90\xa2&\x9c\xcf\xfa\xb5U^\xda\xf1\xdc\xa46R[', b'\xb1g\x16\xb2\x00.\xb8\xa3\xf5\xb0\x18d\x00J\xaf8', b'\x94k\x0c?\x00\xcb\xefb}\x91\x15\xd7\xa8\xf9\x19\xd5', b'\xdc\xafz+\x00\xe9zat\xda\x14G<\x98\xcdw', b'\xab>\x81[\xd3\xb5\xa6p\\\xf2\x18\xc3\x13\xec\xe1#']
[b'comment1%3Dcooki', b'ng%2520MCs%3Buse', b'\x10\x15\n6P\x98\xe68<\x92\xed\x97\xa1\xbaZ\xc2', b';admin=true;%3Bc', b'omment2%3D%2520l', b'ike%2520a%2520po', b'und%2520of%2520b', b'acon\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c']
True
